# BigQuery Pipeline
Google Cloud Datalab, with the *pipeline* subcommand, enables productionizing (i.e. scheduling and orchestrating) notebooks that accomplish ETL with BigQuery and GCS. 



In [38]:
%%bq pipeline --help

usage: %bq pipeline [-h] -n NAME [-d GCS_DAG_BUCKET] [-f GCS_DAG_FILE_PATH]

Creates a GCS/BigQuery ETL pipeline. The cell-body is specified as follows:
  input:
    table | path: <BQ table name or GCS path; both if path->table load is also required>
    schema: <For syntax, refer '%%bq execute'>
    format: {csv (default) | json}
    csv: <This section is relevant only when 'format' is 'csv'>
      delimiter: <The field delimiter to use; default is ','>
      skip: <Number of rows at the top of a CSV file to skip; default is 0>
      strict: <{True | False (default)}; whether to accept rows with missing trailing (or optional) columns>
      quote: <Value used to quote data sections; default is '"'>
    mode: <{append (default) | overwrite}; required if path->table load>
  transformation: <optional; when absent, a direct conversion is done from input (path|table) to output (table|path)>
    query: <name of BQ query defined via "%%bq query --name ...">
  output:
    table | path: <BQ ta

# Setup
We set up a few tutorial-essentials in the following cell. These are garbage-collected in the *Cleanup* cell at the end of this notebook.


We use Airflow (https://airflow.apache.org/start.html) as the underlying technology for orchestrating and scheduling. To set this up, please first run the "Airflow Setup" notebook (under tutorials/BigQuery at https://datalab-alpha.cloud.google.com/docs); it will setup a GCE VM with the Airflow scheduler and the webserver as a long-running process.


Running all the cells in the notebook would ensure that there is a VM instance named "datalab-airflow" in your project. Note: Without the correct setup of the VM above, the *pipeline* subcommand in the cells below will not work (pipelines will not be deployed).

In [0]:
from google.datalab import Context
import datetime
import google.datalab.bigquery as bq
import google.datalab.storage as storage

project = Context.default().project_id

bucket_name = project + '-bq_pipeline'
bucket = storage.Bucket(bucket_name)
bucket.create()

dataset_name = 'bq_pipeline'
dataset = bq.Dataset(dataset_name)
dataset.create()

gcs_dag_bucket_name = project + '-datalab-airflow'
gcs_dag_file_path = 'dags'

# Start and end timestamps for our pipelines. 
start = datetime.datetime.now()
formatted_start = start.strftime('%Y%m%dT%H%M%S')
end = start + datetime.timedelta(minutes=5)




# Building a data transformation pipeline
The *pipeline* subcommand deploys and orchestrates an ETL pipeline. It supports specifying either an existing BQ table or a GCS path (with accompanying schema) as the data *input*, executing a *transformation* with BQ SQL and producing an *output* of the results (again, either a BQ table or a GCS path). This *pipeline* can be executed on a *schedule*. Additionally, *parameters* can be specified to templatize or customize the pipeline.

In [0]:
github_archive = 'githubarchive.month.201801'

In [0]:
%%bq query --name my_pull_request_events
SELECT id, created_at, repo.name FROM input
WHERE actor.login = 'rajivpb' AND type = 'PullRequestEvent'

In [0]:
# We designate the following 'output' for our pipeline. 
results_table = project + '.' + dataset_name + '.' + 'pr_events_' + formatted_start

# Pipeline name is made unique by suffixing a timestamp
pipeline_name = 'github_once_' + formatted_start

In [25]:
%%bq pipeline --name $pipeline_name -d $gcs_dag_bucket_name -f $gcs_dag_file_path
input:
  table: $github_archive
transformation:
  query: my_pull_request_events
output:
  table: $results_table
  mode: overwrite
schedule:
  start: $start
  end: $end
  interval: '@once'
  catchup: True

[2018-01-26 01:47:00,324] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:00,330] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:00,335] {connectionpool.py:207} INFO - Starting new HTTP connection (1): metadata.google.internal
[2018-01-26 01:47:00,382] {connectionpool.py:758} INFO - Starting new HTTPS connection (1): www.googleapis.com


'\nimport datetime\nfrom airflow import DAG\nfrom airflow.operators.bash_operator import BashOperator\nfrom airflow.contrib.operators.bigquery_operator import BigQueryOperator\nfrom airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator\nfrom airflow.contrib.operators.bigquery_to_bigquery import BigQueryToBigQueryOperator\nfrom airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator\nfrom airflow.contrib.operators.gcs_to_bq import GoogleCloudStorageToBigQueryOperator\nfrom google.datalab.contrib.bigquery.operators._bq_load_operator import LoadOperator\nfrom google.datalab.contrib.bigquery.operators._bq_execute_operator import ExecuteOperator\nfrom google.datalab.contrib.bigquery.operators._bq_extract_operator import ExtractOperator\nfrom datetime import timedelta\n\ndefault_args = {\n    \'owner\': \'Google Cloud Datalab\',\n    \'email\': [],\n    \'start_date\': datetime.datetime.strptime(\'2018-01-26T01:46:54\', \'%Y-%m-%d

When the above cell is run, a pipeline is deployed and the results of the query are written into the BQ results table (i.e. $results_table). It could take 5-10 min between when the cell is executed for the result_table to show up. Below, we'll see additional examples for alternate ways of specifying the source, the source-types supported, and for customizing the pipeline.

# Parameterization
The *parameters* section provides the ability to customize the inputs and outputs of the pipeline. These parameters are merged with the SQL query parameters into a list, and are specified in the cell body (along the same lines as the *%bq execute* command, for example). 

In addition to parameters that the users can define, the following mapping keys have been made available for formatting strings and are designed to capture common scenarios around parameterizing the pipeline with the execution timestamp. 

 - '_ds': the date formatted as YYYY-MM-DD
 - '_ts': the full ISO-formatted timestamp YYYY-MM-DDTHH:MM:SS.mmmmmm
 - '_ds_nodash': the date formatted as YYYYMMDD (i.e. YYYY-MM-DD with 'no dashes')
 - '_ts_nodash': the timestamp formatted as YYYYMMDDTHHMMSSmmmmmm (i.e full ISO-formatted timestamp without dashes or colons)
 - '_ts_year': 4-digit year
 - '_ts_month': '1'-'12'
 - '_ts_day': '1'-'31'
 - '_ts_hour': '0'-'23'
 - '_ts_minute': '0'-'59'
 - '_ts_second': '0'-'59'


In [0]:
# The source/input is formatted with the built-in mapping keys _ts_year and 
# _ts_month and these are evaluated (or "bound") at the time of pipeline 
# execution. This could be at some point in the future, or at some point in the 
# "past" in cases where a backfill job is being executed.
github_archive_current_month = 'githubarchive.month.%(_ts_year)s%(_ts_month)s'

# The destination/output is formatted with additional user-defined parameters
# 'project', 'dataset', and 'user'. These are evaluated/bound at the time of 
# execution of the %bq pipeline cell. 
results_table = '%(project)s.%(dataset_name)s.%(user)s_pr_events_%(_ts_nodash)s'

pipeline_name = 'github_parameterized_' + formatted_start

In [0]:
%%bq query --name my_pull_request_events
SELECT id, created_at, repo.name FROM input
WHERE actor.login = @user AND type = 'PullRequestEvent'


In [28]:
%%bq pipeline --name $pipeline_name -d $gcs_dag_bucket_name -f $gcs_dag_file_path
input:
  table: $github_archive_current_month
transformation:
  query: my_pull_request_events
output:
  table: $results_table
  mode: overwrite
parameters:
  - name: user
    type: STRING
    value: 'rajivpb'
  - name: project
    type: STRING
    value: $project
  - name: dataset_name
    type: STRING
    value: $dataset_name
schedule:
  start: $start
  end: $end
  interval: '@once'
  catchup: True

[2018-01-26 01:47:04,047] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:04,054] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:04,059] {connectionpool.py:207} INFO - Starting new HTTP connection (1): metadata.google.internal
[2018-01-26 01:47:04,106] {connectionpool.py:758} INFO - Starting new HTTPS connection (1): www.googleapis.com


'\nimport datetime\nfrom airflow import DAG\nfrom airflow.operators.bash_operator import BashOperator\nfrom airflow.contrib.operators.bigquery_operator import BigQueryOperator\nfrom airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator\nfrom airflow.contrib.operators.bigquery_to_bigquery import BigQueryToBigQueryOperator\nfrom airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator\nfrom airflow.contrib.operators.gcs_to_bq import GoogleCloudStorageToBigQueryOperator\nfrom google.datalab.contrib.bigquery.operators._bq_load_operator import LoadOperator\nfrom google.datalab.contrib.bigquery.operators._bq_execute_operator import ExecuteOperator\nfrom google.datalab.contrib.bigquery.operators._bq_extract_operator import ExtractOperator\nfrom datetime import timedelta\n\ndefault_args = {\n    \'owner\': \'Google Cloud Datalab\',\n    \'email\': [],\n    \'start_date\': datetime.datetime.strptime(\'2018-01-26T01:46:54\', \'%Y-%m-%d

# SQL-based data transformation pipeline for GCS data
*pipeline* also supports specifying GCS paths as both the input (accompanied by a schema) and output, thus completely bypassing the specification of any BQ tables. Garbage collection of all intermediate BQ tables is handled for the user.

In [0]:
gcs_input_path = 'gs://cloud-datalab-samples/cars.csv'
gcs_output_path = 'gs://%(bucket_name)s/all_makes_%(_ts_nodash)s.csv'
pipeline_name = 'gcs_to_gcs_transform_' + formatted_start

In [0]:
%%bq query --name all_makes
SELECT Make FROM input

In [31]:
%%bq pipeline --name $pipeline_name -d $gcs_dag_bucket_name -f $gcs_dag_file_path
input:
  path: $gcs_input_path
  schema:
    - name: Year
      type: INTEGER
    - name: Make
      type: STRING
    - name: Model
      type: STRING
    - name: Description
      type: STRING
    - name: Price
      type: FLOAT
  csv:
    skip: 1
transformation: 
  query: all_makes
output:
  path: $gcs_output_path
parameters:
  - name: bucket_name
    type: STRING
    value: $bucket_name
schedule:
  start: $start
  end:  $end
  interval: '@once'
  catchup: True

[2018-01-26 01:47:08,213] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:08,219] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:08,222] {connectionpool.py:207} INFO - Starting new HTTP connection (1): metadata.google.internal
[2018-01-26 01:47:08,267] {connectionpool.py:758} INFO - Starting new HTTPS connection (1): www.googleapis.com


'\nimport datetime\nfrom airflow import DAG\nfrom airflow.operators.bash_operator import BashOperator\nfrom airflow.contrib.operators.bigquery_operator import BigQueryOperator\nfrom airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator\nfrom airflow.contrib.operators.bigquery_to_bigquery import BigQueryToBigQueryOperator\nfrom airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator\nfrom airflow.contrib.operators.gcs_to_bq import GoogleCloudStorageToBigQueryOperator\nfrom google.datalab.contrib.bigquery.operators._bq_load_operator import LoadOperator\nfrom google.datalab.contrib.bigquery.operators._bq_execute_operator import ExecuteOperator\nfrom google.datalab.contrib.bigquery.operators._bq_extract_operator import ExtractOperator\nfrom datetime import timedelta\n\ndefault_args = {\n    \'owner\': \'Google Cloud Datalab\',\n    \'email\': [],\n    \'start_date\': datetime.datetime.strptime(\'2018-01-26T01:46:54\', \'%Y-%m-%d

# Load data from GCS into BigQuery
*pipeline* can also be used to parameterize and schedule the loading of data from GCS to BQ, i.e the equivalent of the *%bq load* command.

In [0]:
bq_load_results_table = '%(project)s.%(dataset_name)s.cars_load_%(_ts_nodash)s'
pipeline_name = 'load_gcs_to_bq_' + formatted_start

In [33]:
%%bq pipeline --name $pipeline_name -d $gcs_dag_bucket_name -f $gcs_dag_file_path
load:
  path: $gcs_input_path
  schema:
    - name: Year
      type: INTEGER
    - name: Make
      type: STRING
    - name: Model
      type: STRING
    - name: Description
      type: STRING
    - name: Price
      type: FLOAT
  csv:
    skip: 1
  table: $bq_load_results_table
  mode: overwrite
parameters:
  - name: project
    type: STRING
    value: $project
  - name: dataset_name
    type: STRING
    value: $dataset_name
schedule:
  start: $start
  end: $end
  interval: '@once'
  catchup: True

[2018-01-26 01:47:11,013] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:11,019] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:11,023] {connectionpool.py:207} INFO - Starting new HTTP connection (1): metadata.google.internal
[2018-01-26 01:47:11,069] {connectionpool.py:758} INFO - Starting new HTTPS connection (1): www.googleapis.com


'\nimport datetime\nfrom airflow import DAG\nfrom airflow.operators.bash_operator import BashOperator\nfrom airflow.contrib.operators.bigquery_operator import BigQueryOperator\nfrom airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator\nfrom airflow.contrib.operators.bigquery_to_bigquery import BigQueryToBigQueryOperator\nfrom airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator\nfrom airflow.contrib.operators.gcs_to_bq import GoogleCloudStorageToBigQueryOperator\nfrom google.datalab.contrib.bigquery.operators._bq_load_operator import LoadOperator\nfrom google.datalab.contrib.bigquery.operators._bq_execute_operator import ExecuteOperator\nfrom google.datalab.contrib.bigquery.operators._bq_extract_operator import ExtractOperator\nfrom datetime import timedelta\n\ndefault_args = {\n    \'owner\': \'Google Cloud Datalab\',\n    \'email\': [],\n    \'start_date\': datetime.datetime.strptime(\'2018-01-26T01:46:54\', \'%Y-%m-%d

# Extract data from BigQuery into GCS
Similar to load, *pipeline* can also be used to perform the equivalent of the *%bq extract* command.

In [0]:
# This pipeline depends on the output table from the previous pipeline (i.e. the 
# "load") pipeline. So we have this start a wee bit later.
later_start = start + datetime.timedelta(minutes=5)
later_end = later_start + datetime.timedelta(minutes=5)

gcs_extract_path = 'gs://%(bucket_name)s/cars_extract_%(_ts_nodash)s.csv'
pipeline_name = 'extract_bq_to_gcs_' + later_start.strftime('%Y%m%dT%H%M%S')

In [35]:
%%bq pipeline --name $pipeline_name -d $gcs_dag_bucket_name -f $gcs_dag_file_path
extract:
  table: $bq_load_results_table
  path: $gcs_extract_path
  format: csv
  csv:
    delimiter: '#'
parameters:
  - name: bucket_name
    type: STRING
    value: $bucket_name
  - name: project
    type: STRING
    value: $project
  - name: dataset_name
    type: STRING
    value: $dataset_name
schedule:
  start: $later_start
  end:  $later_end
  interval: '@once'
  catchup: True

[2018-01-26 01:47:14,057] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:14,064] {_default.py:280} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2018-01-26 01:47:14,068] {connectionpool.py:207} INFO - Starting new HTTP connection (1): metadata.google.internal
[2018-01-26 01:47:14,115] {connectionpool.py:758} INFO - Starting new HTTPS connection (1): www.googleapis.com


'\nimport datetime\nfrom airflow import DAG\nfrom airflow.operators.bash_operator import BashOperator\nfrom airflow.contrib.operators.bigquery_operator import BigQueryOperator\nfrom airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator\nfrom airflow.contrib.operators.bigquery_to_bigquery import BigQueryToBigQueryOperator\nfrom airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator\nfrom airflow.contrib.operators.gcs_to_bq import GoogleCloudStorageToBigQueryOperator\nfrom google.datalab.contrib.bigquery.operators._bq_load_operator import LoadOperator\nfrom google.datalab.contrib.bigquery.operators._bq_execute_operator import ExecuteOperator\nfrom google.datalab.contrib.bigquery.operators._bq_extract_operator import ExtractOperator\nfrom datetime import timedelta\n\ndefault_args = {\n    \'owner\': \'Google Cloud Datalab\',\n    \'email\': [],\n    \'start_date\': datetime.datetime.strptime(\'2018-01-26T01:46:54\', \'%Y-%m-%d

#Cleanup

In [0]:
# Delete the contents of the GCS bucket, the GCS bucket itself, and the BQ 
# dataset. Uncomment the lines and execute them.
#!gsutil rm -r gs://$bucket_name
#!gsutil rb gs://$bucket_name
#!bq rm -r -f $dataset_name

In [37]:
# This just verifies that cleanup actually worked. 

#Should show an error message like BucketNotFoundException: 404 gs://...
!gsutil ls gs://$bucket_name
!gsutil ls gs://$gcs_dag_bucket_name
  
#Should show an error message like BigQuery error in ls operation: Not found ...
!bq ls $dataset_name


gs://cloud-ml-dev-datalab-airflow/dags/
